## Agent Inventroy Script

In [ ]:
from decimal import *
modified_by_id=99
agent_inventry_obj = AgentInventory.objects.filter()
# print(agent_inventry_obj.date_of_receipt)
for item in agent_inventry_obj:
#     print(item.agent_id)
#     print(item.date_of_receipt)
    if not AgentWallet.objects.filter(agent_id=item.agent_id).exists():
        updated_amount = 0 - (item.quantity_at_receipt * item.combo_issue_request.input_combo.price)
        old_balance = 0
        print("type : ", updated_amount)
        AgentWallet.objects.create(
            agent_id=item.agent_id,
            current_balance=updated_amount,
            modified_by_id=modified_by_id,
        )
        new_balance = updated_amount
    else:
        old_balance = AgentWallet.objects.get(agent_id=item.agent_id).current_balance
        updated_amount = old_balance - item.quantity_at_receipt * item.combo_issue_request.input_combo.price
        AgentWallet.objects.filter(agent_id=item.agent_id).update(current_balance=Decimal(updated_amount), modified_by=modified_by_id)
        new_balance = updated_amount

    agent_transaction_log = AgentTransactionLog(
        date=item.date_of_receipt,
        transaction_direction_id=2,  # ccgb to agent wallet when wallet adjustment - positive
        agent_id=item.agent_id,
        data_entered_by_id=99,
        amount= item.quantity_at_receipt * item.combo_issue_request.input_combo.price,
        transaction_id="1234",
        transaction_mode_id=1,
        transaction_approval_status_id=1,
        wallet_balance_before_this_transaction=old_balance,
        wallet_balance_after_this_transaction=new_balance,
        description="it will be added as a positive amount in agent wallet as a commision for wallet adjustment",
        modified_by_id=99,
    )
    agent_transaction_log.save()

    input_distribution_transaction_map_obj = InputDistributionTransactionMap(
                                            agent_inventory_id = item.id,
                                            transaction_log_id = agent_transaction_log.id)
    input_distribution_transaction_map_obj.save()

    print(1)


## Agent Retrun log script (only if return is active)

In [ ]:
combo_return_request_obj = ComboReturnRequest.objects.filter(season_id=3)
for item in combo_return_request_obj:
    print(item.agent.id)


combo_return_request_obj = ComboReturnRequest.objects.filter(id=id)
    combo_return_request_label_map_obj = ComboReturnRequestLabelMap.objects.filter(return_label__status_id=2, combo_return_request_id=id)

    combo_return_request_label_map_obj_count = combo_return_request_label_map_obj.count()

    if not AgentWallet.objects.filter(agent_id=combo_return_request_obj.agent_id).exists():
        updated_amount = round(combo_return_request_label_map_obj_count * combo_return_request_label_map_obj.return_label.agent_inventory.combo_issue_request.input_combo.price) #price need update
        old_balance = 0
        print("type : ", updated_amount)
        AgentWallet.objects.create(
            agent_id=combo_return_request_obj.agent_id,
            current_balance=updated_amount,
            modified_by_id=modified_by_id,
        )
        new_balance = updated_amount

    else:
        old_balance = AgentWallet.objects.get(agent_id=combo_return_request_obj.agent_id).current_balance
        updated_amount = old_balance + combo_return_request_label_map_obj_count * combo_return_request_label_map_obj.return_label.agent_inventory.combo_issue_request.input_combo.price
        AgentWallet.objects.filter(agent_id=combo_return_request_obj.agent_id).update(current_balance=Decimal(updated_amount), modified_by=modified_by_id)
        new_balance = updated_amount

    agent_transaction_log = AgentTransactionLog(
        date=datetime.datetime.today(),
        transaction_direction_id=4,  # agent seed return
        agent_id=combo_return_request_obj.agent.id,
        data_entered_by_id=modified_by_id,
        amount= combo_return_request_label_map_obj_count * combo_return_request_label_map_obj.return_label.agent_inventory.combo_issue_request.input_combo.price, 
        transaction_id="1234",
        transaction_mode_id=1,
        transaction_approval_status_id=1,
        wallet_balance_before_this_transaction=old_balance,
        wallet_balance_after_this_transaction=new_balance,
        description="money adjustment to wallet - adding money to wallet(positive)",
        modified_by_id=modified_by_id,
    )
    agent_transaction_log.save()

    input_return_transaction_log_obj = InputReturnTransactionLog(
                                        transaction_log_id = agent_transaction_log.id,
                                        combo_return_request_id = combo_return_request_obj.id
                                        )
    input_return_transaction_log_obj.save()
    print(1)

## Procurement Transaction Log script

In [ ]:
import datetime
from decimal import *

modified_by_id = 99
procurement_obj = Procurement.objects.filter()

for item in procurement_obj:
    #     print(item.procurement_group.payment_to_agent)
    if not AgentWallet.objects.filter(agent_id=item.procurement_group.agent_id).exists():
        updated_amount = item.procurement_group.payment_to_wallet
        old_balance = 0
        print("type : ", updated_amount)
        AgentWallet.objects.create(
            agent_id=item.procurement_group.agent_id,
            current_balance=updated_amount,
            modified_by_id=modified_by_id,
        )
        new_balance = updated_amount
    else:
        old_balance = AgentWallet.objects.get(agent_id=item.procurement_group.agent_id).current_balance
        updated_amount = old_balance + item.procurement_group.payment_to_wallet
        AgentWallet.objects.filter(agent_id=item.procurement_group.agent_id).update(current_balance=Decimal(updated_amount), modified_by_id=modified_by_id)
        new_balance = updated_amount
    agent_transaction_log = AgentTransactionLog(
        date=datetime.datetime.today(),
        transaction_direction_id=3,  
        agent_id=item.procurement_group.agent_id,
        data_entered_by_id=modified_by_id,
        amount= item.procurement_group.payment_to_wallet, 
        transaction_id="1234",
        transaction_mode_id=1,
        transaction_approval_status_id=1,
        wallet_balance_before_this_transaction=old_balance,
        wallet_balance_after_this_transaction=new_balance,
        description="it will be given to agent hand",
        modified_by_id=modified_by_id,
    )
    agent_transaction_log.save()
    print('payment_to_agent_saved_log')

    input_procurement_transaction_log_obj = InputProcurementTransactionLog(
                                            transaction_log_id = agent_transaction_log.id,
                                            procurement_id = item.id)
    input_procurement_transaction_log_obj.save()

    agent_transaction_log = AgentTransactionLog(
        date=datetime.datetime.today(),
        transaction_direction_id=2,  
        agent_id=item.procurement_group.agent_id,
        data_entered_by_id=modified_by_id,
        amount= item.procurement_group.payment_to_agent, 
        transaction_id="1234",
        transaction_mode_id=1,
        transaction_approval_status_id=1,
        wallet_balance_before_this_transaction=old_balance,
        wallet_balance_after_this_transaction=new_balance,
        description="it will be given to agent wallet ",
        modified_by_id=modified_by_id,
    )
    agent_transaction_log.save()

    input_procurement_transaction_log_obj = InputProcurementTransactionLog(
                                            transaction_log_id = agent_transaction_log.id,
                                            procurement_id = item.id)
    input_procurement_transaction_log_obj.save()
    print('saved_saved')